In [ ]:
# ===========================================
# 02_preprocess_patients.ipynb
# Generación de tensores X (CT + máscaras) y Y (dosis)
# ===========================================

import os
import sys

# Añadimos la carpeta src al path para poder importar nuestros módulos
sys.path.append("../src")

import numpy as np

from preprocessing import prepare_patient, list_patients

# -------------------------------------------
# CONFIGURACIÓN
# -------------------------------------------

data_root = "../data_raw"
out_dir = "../data_processed"

# Lista de estructuras que quieres como canales además del CT
# Ajusta según tu sitio anatómico y cómo se llaman en tus RTSTRUCT
roi_order = [
    "PTV",
    "CTV",
    "GTV",
    "Rectum",
    "Bladder",
    "FemoralHead_L",
    "FemoralHead_R"
]

print("Data root:      ", data_root)
print("Output dir:     ", out_dir)
print("ROI order (canales):", roi_order)

# -------------------------------------------
# LISTAR PACIENTES DISPONIBLES
# -------------------------------------------

patients = list_patients(data_root)
print("\nPacientes encontrados en data_root:")
for pid in patients:
    print("  -", pid)

# Si quieres, puedes limitar a algunos pacientes al inicio:
# patients = patients[:5]

# -------------------------------------------
# PREPROCESAR TODOS LOS PACIENTES
# -------------------------------------------

saved_paths = []
skipped_patients = []

for pid in patients:
    print("\n===============================")
    print(f"Procesando paciente: {pid}")
    print("===============================")
    out_path = prepare_patient(data_root, pid, roi_order, out_dir)
    if out_path is not None:
        saved_paths.append(out_path)
    else:
        skipped_patients.append(pid)

print("\n---------------------------------")
print("Resumen del preprocessing")
print("---------------------------------")
print(f"Pacientes procesados y guardados: {len(saved_paths)}")
for p in saved_paths:
    print("  ✅", p)

if skipped_patients:
    print(f"\nPacientes saltados (formas CT/DOSE distintas u otros problemas): {len(skipped_patients)}")
    for pid in skipped_patients:
        print("  ⚠️", pid)
else:
    print("\nNo se saltó ningún paciente.")

# -------------------------------------------
# INSPECCIÓN RÁPIDA DE UN ARCHIVO .npz
# -------------------------------------------

if saved_paths:
    sample_path = saved_paths[0]
    print("\nEjemplo de archivo guardado:", sample_path)
    data = np.load(sample_path, allow_pickle=True)

    X = data["X"]
    Y = data["Y"]
    ct_spacing = data["ct_spacing"]
    dose_spacing = data["dose_spacing"]
    roi_order_np = data["roi_order"]

    print("\n--- Info del ejemplo ---")
    print("X shape (C,Z,Y,X):", X.shape)
    print("Y shape (Z,Y,X):   ", Y.shape)
    print("CT spacing:        ", ct_spacing)
    print("Dose spacing:      ", dose_spacing)
    print("ROI order guardado:", roi_order_np)
